# Latent Dirichlet Allocation on MIT projects 

## First a simple example
https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

In [1]:
# corpus
doc1 = "Music is good to hear because it can make you happy and you can play it too"
doc2 = "it's bad working all your life and have no time for love"
doc3 = "Enjoying music is good and relaxing. it is best to play it but you need concentration"
doc4 = "I program easy stuff. I enjoy it staying in front of the computer all the time"
doc5 = "politics is corrupt, this is a rigged system. My favorite dictator would put everything in order"
doc_complete = [doc1, doc2, doc3, doc4, doc5]

# Tokenize
from nltk.corpus import stopwords 
import string
#from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
#lemma = WordNetLemmatizer()
pstemmer = PorterStemmer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    #normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    normalized = " ".join(pstemmer.stem(word) for word in punc_free.split())
    return normalized
doc_clean = [clean(doc).split() for doc in doc_complete]

# Preparing Document-Term Matrix containing (term ID, term frequency) pairs
import gensim
from gensim import corpora
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
doc_clean[0:]

ImportError: No module named gensim

In [ ]:
print(dictionary.token2id['music'])

In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)
topics = ldamodel.print_topics(num_topics=3, num_words=3)
for t in topics: print(t)


## Now back to the MIT abstracts

In [ ]:
import json
#Import dictionary with urls from file
with open('abs_dict.json', 'r') as outfile:
    abs_dict = json.load(outfile)
#If the file is not there get it from the script
if 'abs_dict' not in locals():
    import gather
    abs_dict = gather.main()
    with open('abs_dict.json', 'w') as outfile:
        json.dump(abs_dict, outfile)

In [ ]:
#Read the abstracts that I previously scraped and stored in the /abstract/ folder (find more details in README.md)
import scrape
corpus = scrape.readAbstracts()
corpus

In [ ]:
#Create documents list with tokenized text
# Tokenize
from nltk.corpus import stopwords 
import string
import re
stop = set(stopwords.words('english')) #NB a set is more efficient than a list
exclude = set(string.punctuation) 
#from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
#pstemmer = PorterStemmer()
def clean_text(doc, skipWords=[]):
    for w in skipWords:
        doc = doc.replace(w,'')
    #Remove non-letters and convert to lower case
    #Remove non unicode (accents etc). Remove word that can starts with non letter and ends with non letter. 
    #"1st" "a2" "1a3" safe, "1" "123" esta' " not
    letters_only = re.sub(r'\W|\b\w*\d\b', ' ', doc.lower())
    #Split into individual words and remove stop words
    #This can be done in TfidfVectorizer/CountVectorizer but ok..
    stop_free = " ".join([i for i in letters_only.split() if i not in stop])
    #Remove punctuations should not be needed after letters_only but who knows
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    #Stemming/Lemmatizing
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
documents = [clean_text(doc) for doc in corpus]

### Fit [LDA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)
I am using 2-grams (i.e. features composed of two words). Five topics (N=5) can be discerned using a quite low cutoff for the words appearing in more than 95% of the documets (max_df=0.95). 

I played with the parameters and the topics seem relatively stable. Based on the pyLDAvis plot, using 1-grams leads to two partially overlapping topics with N=5. This is the reason why I chose 2-grams, although I only see 2-gram ("lower bound") within the most salient features. The fit using for 1- and 2-grams are sensitive on the maximum number of features (here max_features=1000) but seem stable when changing max_df from 50% to 95%. The fit with 1-grams on the other hand needs lower value for max_features to show a better qualitative fit (i.e. less overlap in topics).


<!--2-grams: 5 topics, max_features=1000 is well separated. a few are similar to the pdf. better than 1-gram. I see one 2-gram only ("lower bound"). max_features=1700 begins to overlap. max_df=0.95, 0.75, 0.5 have very similar fit

1-gram: 5 topics, max_features=700 is better than 1000, though overlap 2 topics with both max_df=0.95 and 0.65
-->

In [ ]:
# Learn the vocabulary dictionary and return term-document matrix.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(analyzer='word', stop_words='english', #token_pattern = r'\b[a-zA-Z]{3,}\b',#strip_accents='ascii', tokenizer=lemma.lemmatize, lowercase=True, 
                        ngram_range=(1,2), max_df=0.95, min_df=5, max_features=1000)
#Fit Transform - Learn the vocabulary dictionary and return term-document matrix
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
no_topics = 5
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50., random_state=0, n_jobs=2)
lda.fit(tf)
# Display the terms in the topics
#tf_vectorizer.get_feature_names(): map feature indices -> feature name
#lda.components_[i,j] (topic word distribution): "weights" of terms j in topic i
def display_topics(model, feature_names, no_top_words):
    for ind, topic in enumerate(model.components_):
        print "Topic %d:" % (ind)
        print " ".join([feature_names[i]+' -'
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
print
print "LDA:"
display_topics(lda, tf_feature_names, no_top_words = 10)
#Plot with pyLDAvis - see http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb
import pyLDAvis.sklearn, pyLDAvis
pyLDAvis.enable_notebook();
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, ImageColorGenerator

def plot_wordcloud(components,feature_names,numbest = 25):
    for t in range(len(components)):
        best_terms = dict([(feature_names[i],components[t,i]) for i in components[t].argsort()[:-numbest-1:-1].tolist()])
        # Generate a word cloud image
        wordcloud = WordCloud(width=400, height=200, margin=20, background_color='black', max_font_size=50) #random_state=i, 
        wordcloud.generate_from_frequencies(best_terms)
        # Display the generated image:
        # the matplotlib way:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(12, 6))
        plt.axis("off")
        plt.imshow(wordcloud, interpolation='bilinear')

plot_wordcloud(lda.components_,tf_feature_names,25)

In [ ]:
# Preparing Document-Term Matrix
import gensim
from gensim import corpora
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(corpus)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

In [ ]:
topics = ldamodel.print_topics(num_topics=5, num_words=7)
for t in topics: print t